## Implementation of RNN from scratch for Artificial Intelligence Comp.5430

In [21]:
import numpy as np
from Modules.rnnmodel import RNN_Scratch
from Modules.rnn_utils.utilities import SGD, one_hot_encoding
import pandas as pd
import glob

In [22]:
np.random.seed(1)

In [23]:
# Read in data
df = pd.read_csv('../../archive/A.csv')

df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1999-11-18,27.896016,30.654965,24.523970,26.976368,62546380,0.0,0.0
1,1999-11-19,26.324950,26.363270,24.409017,24.753885,15234146,0.0,0.0
2,1999-11-22,25.328663,26.976368,24.562289,26.976368,6577870,0.0,0.0
3,1999-11-23,26.056724,26.746461,24.523975,24.523975,5975611,0.0,0.0
4,1999-11-24,24.600619,25.711862,24.523981,25.175400,4843231,0.0,0.0
...,...,...,...,...,...,...,...,...
5692,2022-07-06,120.800003,122.339996,120.559998,121.180000,1642300,0.0,0.0
5693,2022-07-07,120.989998,123.220001,120.620003,123.150002,1221200,0.0,0.0
5694,2022-07-08,120.959999,123.209999,120.959999,122.510002,1267300,0.0,0.0
5695,2022-07-11,121.610001,123.459999,121.010002,123.120003,1304400,0.0,0.0


## Model

In [24]:
# Define Model
num_epochs = 10
input_dim = 1
output_dim = 11
hidden_dim = 64

# initialize and define the model hyperparamaters
model = RNN_Scratch(input_dim, output_dim, hidden_dim)
optim = SGD(lr=0.001)
costs = []

## Train

In [25]:
from Modules.train import train_model
from Modules.dataset_class import TimeSeriesDataset
from Modules.preprocess import process_data
from torch.utils.data import DataLoader
import torch
def train_on_file(file, model, num_epochs, loss_function, optimizer, device):

    print("PROCESSING "+ file)
    _, _, X_train, X_test, y_train, y_test, _ = process_data(file)
    train_dataset = TimeSeriesDataset(X_train, y_train)
    test_dataset = TimeSeriesDataset(X_test, y_test)
    X_train.shape, X_test.shape, y_train.shape, y_test.shape

    # create batches
    batch_size = 16

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model = train_model(num_epochs, model, loss_function, optimizer, train_loader,test_loader, device)
    return model


In [26]:
def train_all(files, input_model, num_epochs, loss_function, optimizer, device):
    model = input_model
    save = 1
    for file in files:
        model = train_on_file(file, model, num_epochs, loss_function, optimizer, device)
        if save % 5 == 0:
            torch.save(model, 'checkpoints/forecast'+str(save)+'.pt')
        save += 1

    torch.save(model, 'forecast_final.pt')

In [27]:
files = sorted(glob.glob("archive/*.csv"))[:10]
#files = glob.glob("../..archive/*.csv")
#random.shuffle(files)

files_list_path = "../../checkpoints/filenames.txt"
with open(files_list_path, 'w') as file:
    # Write each element of the string array to the file
    for item in files:
        file.write(item + '\n')



train_all(files, model, num_epochs, loss_function, optimizer, device)

NameError: name 'random' is not defined